In [1]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5a41bbee-cd36-4d18-8457-e2fda89ba971;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 959ms :: artifacts dl 39ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-run

In [3]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

+---------+
|namespace|
+---------+
|  default|
|     demo|
+---------+



In [4]:
spark.sql(
    """
    DROP DATABASE IF EXISTS demo CASCADE
    """
)

DataFrame[]

In [5]:
# you can choose any location in HDFS, just be organized 
# Your data lake will grow with time and will become a swamp
spark.sql(
    """
    CREATE DATABASE demo LOCATION 'hdfs://hdfs-nn:9000/warehouse/demo.db/'
    """
)

DataFrame[]

In [6]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

+---------+
|namespace|
+---------+
|  default|
|     demo|
+---------+



In [7]:
spark.sql(
    """
    SHOW TABLES FROM demo
    """
).show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [8]:
spark.sql(
    """
    DROP TABLE IF EXISTS demo.text_table
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE demo.text_table (
        customer_name VARCHAR(50),
        month INT,
        year INT,
        sales INT
    )
    ROW FORMAT DELIMITED
    FIELDS TERMINATED BY ';'
    STORED AS TEXTFILE
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/demo.db/text_table/'
    """
)

# tblproperties('skip.header.line.count'='1')
# can be used for csvs with header
# but spark sql cannot understand that at the moment, while reading the data using sql queries
# so just when creating hive tables backed up by csvs, avoid headers
# in the project we wont use hive text tables, so all good.

21/10/31 22:56:30 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [9]:
spark.sql(
    """
    DROP TABLE IF EXISTS demo.parquet_table
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE demo.parquet_table (
        customer_name VARCHAR(50),
        month INT,
        sales INT
    )
    STORED AS PARQUET
    PARTITIONED BY (
        year INT
    )
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/demo.db/parquet_table/'
    """
)

DataFrame[]

In [10]:
spark.sql(
    """
    DROP TABLE IF EXISTS demo.deltalake_table
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE demo.deltalake_table (
        customer_name VARCHAR(50),
        month INT,
        sales INT
    )
    USING DELTA
    PARTITIONED BY (
        year INT
    )
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/demo.db/deltalake_table/'
    """
)

21/10/31 22:56:48 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `demo`.`deltalake_table` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [11]:
spark.sql(
    """
    SHOW TABLES FROM demo
    """
).show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|    demo|deltalake_table|      false|
|    demo|  parquet_table|      false|
|    demo|     text_table|      false|
+--------+---------------+-----------+



In [12]:
# Let's look into HDFS

In [13]:
spark.sql(
    """
    SELECT *
    FROM demo.text_table
    """
).show()

+-------------+-----+----+-----+
|customer_name|month|year|sales|
+-------------+-----+----+-----+
+-------------+-----+----+-----+



In [14]:
spark.sql(
    """
    SELECT *
    FROM demo.parquet_table
    """
).show()

+-------------+-----+-----+----+
|customer_name|month|sales|year|
+-------------+-----+-----+----+
+-------------+-----+-----+----+



In [15]:
spark.sql(
    """
    SELECT *
    FROM demo.deltalake_table
    """
).show()

+-------------+-----+-----+----+
|customer_name|month|sales|year|
+-------------+-----+-----+----+
+-------------+-----+-----+----+



In [16]:
spark.sql(
    """
    DESCRIBE FORMATTED demo.text_table
    """
).toPandas()

col_name  \
0                  customer_name   
1                          month   
2                           year   
3                          sales   
4                                  
5   # Detailed Table Information   
6                       Database   
7                          Table   
8                          Owner   
9                   Created Time   
10                   Last Access   
11                    Created By   
12                          Type   
13                      Provider   
14              Table Properties   
15                      Location   
16                 Serde Library   
17                   InputFormat   
18                  OutputFormat   
19            Storage Properties   
20            Partition Provider   

                                            data_type comment  
0                                         varchar(50)    None  
1                                                 int    None  
2                                                 int    None  
3                                                 int    None  
4                                                              
5                                                              
6                                                demo          
7                                          text_table          
8                                              jovyan          
9                        Sun Oct 31 22:56:30 UTC 2021          
10                                            UNKNOWN          
11                                        Spark 3.1.2          
12                                           EXTERNAL          
13                                               hive          
14                 [transient_lastDdlTime=1635720990]          
15   hdfs://hdfs-nn:9000/warehouse/demo.db/text_table          
16  org.apache.hadoop.hive.serde2.lazy.LazySimpleS...          
17           org.apache.hadoop.mapred.TextInputFormat          
18  org.apache.hadoop.hive.ql.io.HiveIgnoreKeyText...          
19            [serialization.format=;, field.delim=;]          
20                                            Catalog

In [17]:
spark.sql(
    """
    DESCRIBE FORMATTED demo.parquet_table
    """
).toPandas()

col_name  \
0                  customer_name   
1                          month   
2                          sales   
3                           year   
4        # Partition Information   
5                     # col_name   
6                           year   
7                                  
8   # Detailed Table Information   
9                       Database   
10                         Table   
11                         Owner   
12                  Created Time   
13                   Last Access   
14                    Created By   
15                          Type   
16                      Provider   
17              Table Properties   
18                      Location   
19                 Serde Library   
20                   InputFormat   
21                  OutputFormat   
22            Storage Properties   
23            Partition Provider   

                                            data_type  comment  
0                                         varchar(50)     None  
1                                                 int     None  
2                                                 int     None  
3                                                 int     None  
4                                                               
5                                           data_type  comment  
6                                                 int     None  
7                                                               
8                                                               
9                                                demo           
10                                      parquet_table           
11                                             jovyan           
12                       Sun Oct 31 22:56:30 UTC 2021           
13                                            UNKNOWN           
14                                        Spark 3.1.2           
15                                           EXTERNAL           
16                                               hive           
17                 [transient_lastDdlTime=1635720990]           
18  hdfs://hdfs-nn:9000/warehouse/demo.db/parquet_...           
19  org.apache.hadoop.hive.ql.io.parquet.serde.Par...           
20  org.apache.hadoop.hive.ql.io.parquet.MapredPar...           
21  org.apache.hadoop.hive.ql.io.parquet.MapredPar...           
22                           [serialization.format=1]           
23                                            Catalog

In [18]:
spark.sql(
    """
    DESCRIBE FORMATTED demo.deltalake_table
    """
).toPandas()

col_name  \
0                  customer_name   
1                          month   
2                          sales   
3                           year   
4                                  
5                 # Partitioning   
6                         Part 0   
7                                  
8   # Detailed Table Information   
9                           Name   
10                      Location   
11                      Provider   
12              Table Properties   

                                            data_type comment  
0                                              string          
1                                                 int          
2                                                 int          
3                                                 int          
4                                                              
5                                                              
6                                                year          
7                                                              
8                                                              
9                                demo.deltalake_table          
10  hdfs://hdfs-nn:9000/warehouse/demo.db/deltalak...          
11                                              delta          
12  [Type=EXTERNAL,delta.minReaderVersion=1,delta....

In [19]:
# Let's put the files into HDFS

In [20]:
spark.sql(
    """
    SELECT *
    FROM demo.text_table
    """
).toPandas()

Empty DataFrame
Columns: [customer_name, month, year, sales]
Index: []

In [21]:
# recover partitions is needed so that the Hive Metastore (Catalog)
# is updated. Otherwise Hive and the querying engines do not know
# that there are new parittions in the partitioned table.
spark.catalog.recoverPartitions("demo.parquet_table")

spark.sql(
    """
    SELECT *
    FROM demo.parquet_table
    """
).show()

+-------------+-----+-----+----+
|customer_name|month|sales|year|
+-------------+-----+-----+----+
+-------------+-----+-----+----+



In [22]:
spark.sql(
    """
    SELECT *
    FROM demo.deltalake_table
    """
).show()

+-------------+-----+-----+----+
|customer_name|month|sales|year|
+-------------+-----+-----+----+
+-------------+-----+-----+----+



In [23]:
spark.stop()